<a href="https://www.kaggle.com/code/zaferturan/kira-tahmini?scriptVersionId=195032560" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Bu çalışmada [antalya_kiralik_ev.csv](https://www.kaggle.com/antalyakiralyontem/antalya-kiralik-ev) verisi kullanılmıştır. Veri manipülasyonu, veri görselleştirme ve veri analizi yapmak için [pandas](https://pandas.pydata.org/) ve [seaborn](https://seaborn.pydata.org/) kütüphanelerini kullanılarak yapılmıştır.

## Kütüphaneleri Yükle ##

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Veriyi Yükle ##

In [ ]:
df=pd.read_csv("/kaggle/input/antalya-muratpaa-daire-kira-cretleri-2024-ocak/antalya_kiralik_ev.csv")

## Veri Ön İşleme 

## Keşifsel Veri Analiz (EDA) 

In [ ]:
#ilk 10 satırı listeler.
df.head(10)

In [ ]:
df.describe()

## Unnamed: 0 Kolonunu Sil ##

In [ ]:
df.drop(["Unnamed: 0"],axis = 1, inplace = True)


In [ ]:
df.info()

In [ ]:
df.nunique() #unique değişken sayısı


In [ ]:
df[df['sahibi']==1].groupby('mahalle')['sahibi'].count().sort_values(ascending=False)

In [ ]:
df_kor=df[['fiyat','brut_alan_m2','net_alan_m2','bina_kat_sayisi','banyo_sayisi','balkon','asansor','esya_durumu','site_icinde','aidat','depozito','sahibi']]
outliers=df_kor.quantile(q=.99)
outliers

In [ ]:
outliers = df[['fiyat']].quantile(q=.99) # üst sınır belirliyoruz
outliers #üst sınır

In [ ]:
df[df['fiyat']==df['fiyat'].max()]

In [ ]:
df_non_outliers=df[df['fiyat']<=outliers['fiyat']]
df_non_outliers['fiyat'].max()

In [ ]:
df_non_outliers.info()

In [ ]:
df_non_outliers['banyo_sayisi']=df_non_outliers['banyo_sayisi'].astype('category')
df_non_outliers['balkon']=df_non_outliers['balkon'].astype('category')
df_non_outliers['asansor']=df_non_outliers['asansor'].astype('category')
df_non_outliers['esya_durumu']=df_non_outliers['esya_durumu'].astype('category')
df_non_outliers['site_icinde']=df_non_outliers['site_icinde'].astype('category')
df_non_outliers['sahibi']=df_non_outliers['sahibi'].astype('category')


In [ ]:
df_non_outliers.groupby('mahalle').agg({'fiyat':'mean','net_alan_m2':'mean','aidat':'mean'}).sort_values(by='fiyat',ascending=False)
#mahallelerin fiyat ortalamalarını bulalım

In [ ]:
df['mahalle'].value_counts(normalize=True)
#mahallelerin oranını bulalım

In [ ]:
df['fiyat'].plot.hist(bins=20)
#fiyat histogramını oluşturalım

In [ ]:
df.describe(include="all").T
#tum değişkenlerin istatiksel bilgilerini bulalım

In [ ]:
df.isnull().sum()
#eksik değişkenlerin sayısını bulalım

In [ ]:
numeric_columns=df.select_dtypes(include=['number'])
numeric_columns
#sayısal değişkenleri bulalım

In [ ]:
numeric_columns.corr()
#sayısal değişkenlerin korelasyonunu bulalım

In [ ]:
df['fiyat'].mean()
#fiyat ortalamasını bulalım

In [ ]:
df['fiyat'].median()
#fiyat medyanını bulalım

In [ ]:
df.plot.scatter(x='brut_alan_m2', y='fiyat', alpha=0.5)
plt.title('Brüt Alan vs Fiyat')
plt.show()
#fiyatın brut alanın korelasyonunu bulalım

In [ ]:
df.groupby('mahalle')['fiyat'].mean().plot.bar()
plt.title('Mahallelere Göre Ortalama Fiyat')
plt.ylabel('Ortalama Fiyat')
plt.show()
#mahallelerin fiyat ortalamasını bulalım#

In [ ]:
df['fiyat'].plot.kde()
plt.title('Fiyat Dağılımı ve Yoğunluğu')
plt.show()
#fiyatın dagılımını ve yogunlukunu bulalım

In [ ]:
pivot_table = df.pivot_table(values='fiyat', index='mahalle', columns='isitma_turu', aggfunc='mean')
pivot_table
#isitma türüne göre fiyatın ortalamasını bulalım

In [ ]:
df_non_outliers['dairenin_bulundugu_kat'].value_counts(), df_non_outliers['mahalle'].value_counts()
#dairenin bulundugu kattaki değişkenlerin kategorik olup olmadıgını kontrol edelim#

In [ ]:
from sklearn.preprocessing import LabelEncoder
ss = LabelEncoder()
df_non_outliers['mahalle'] = ss.fit_transform(df_non_outliers['mahalle'])
df_non_outliers['dairenin_bulundugu_kat'] = ss.fit_transform(df_non_outliers['dairenin_bulundugu_kat'])
#mahalle ve dairenin bulundugu kattaki değişkenlerin kategorik olup olmadıgını kontrol edelim#

## ML Modelleme

In [ ]:
from sklearn.model_selection import train_test_split

X = df_non_outliers.drop('fiyat',axis=1)
y = df_non_outliers['fiyat']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2,random_state=42)
#verileri train ve test olarak ayıralım

In [ ]:
#modelleme ve en uygun modeli bulalım
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

def compare_models(X_train,X_test,y_train, y_test):
    
    models = [
        LinearRegression(),
        Ridge(),
        Lasso(),
        DecisionTreeRegressor(),
        RandomForestRegressor(),
        SVR(),
        XGBRegressor(),
        LGBMRegressor(verbose=-1),
        CatBoostRegressor(verbose=False)
    ]
    
    results = []
    for model in models:
        model_name = model.__class__.__name__

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        rmse = mean_squared_error(y_test, y_pred, squared=False)

        r2 = r2_score(y_test, y_pred)
        scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
        rmse_cv = np.sqrt(-scores.mean())
        print(f"{model_name} = RMSE: {rmse}, RMSEcv : {rmse_cv}, r2 Score: {r2}")

In [ ]:
print(X_train.dtypes)


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Veri çerçevesini yükleyin (örnek olarak df_non_outliers kullanılıyor)
# df_non_outliers = pd.read_csv('path_to_your_file.csv')

# LabelEncoder'ı başlatın
le = LabelEncoder()

# Kategorik sütunları dönüştürün
df_non_outliers['oda_sayisi'] = le.fit_transform(df_non_outliers['oda_sayisi'])
df_non_outliers['bina_yas'] = le.fit_transform(df_non_outliers['bina_yas'])
df_non_outliers['isitma_turu'] = le.fit_transform(df_non_outliers['isitma_turu'])
df_non_outliers['otopark'] = le.fit_transform(df_non_outliers['otopark'])

# veya pd.get_dummies() ile dönüştürün
df_non_outliers = pd.get_dummies(df_non_outliers, columns=['oda_sayisi', 'bina_yas', 'isitma_turu', 'otopark'], drop_first=True)

# Verinin türlerini kontrol edin
print(df_non_outliers.dtypes)


In [ ]:
X_train = X_train.dropna()
X_test = X_test.dropna()


In [ ]:
df['fiyat'].describe()


In [ ]:
# Her şehirde kaç şube var?
df['fiyat'].value_counts()


In [ ]:
# veri dağılımını kontrol et
df.hist(bins=20, figsize=(15, 10))
plt.show()

In [ ]:
# Fiyat dağılımını kontrol et
sns.histplot(df['fiyat'], bins=20, kde=True)
plt.show()

## Veri Görselleştirme ##

In [ ]:
df['mahalle'].value_counts(normalize=True)

In [ ]:
df_non_outliers=df[['fiyat','brut_alan_m2','net_alan_m2','bina_kat_sayisi','depozito','banyo_sayisi']]
df_non_outliers.plot()

## Fiyat Görselleştirme ##

In [ ]:
df['fiyat'].plot.hist(bins=20)


## Betimsel İstatistikler ##

In [ ]:
# veri setinin betimsel istatiklerini veriyor.
df.describe(include="all")

In [ ]:
df.describe(include="all").T

## Eksik Veri Kontrolü ##

In [ ]:
df.isnull().sum()

In [ ]:
df['isitma_turu'].value_counts().plot.bar()

In [ ]:
df['fiyat'].value_counts().plot(kind='pie',autopct='%1.1f%%')


## Model Eğitimi

In [ ]:
# ML Modelleme
# Kütüphaneleri yükle
# veri setini bölmek için
from sklearn.model_selection import train_test_split

# ölçeklendirme(Veri ön işleme)
from sklearn.preprocessing import StandardScaler

#linear regression(modelleme)
from sklearn.linear_model import LinearRegression

# değerlendirme
from sklearn.metrics import mean_squared_error,r2_score

In [ ]:
from sklearn.model_selection import train_test_split

X = df_non_outliers.drop('fiyat',axis=1)
y = df_non_outliers['fiyat']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2,random_state=43)
#verileri train ve test olarak ayıralım

In [ ]:
# Kategorik verilerin sayısal verilere dönüştürülmesi
df = pd.get_dummies(df, drop_first=True)

In [ ]:
#modelleme ve en uygun modeli bulalım
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

def compare_models(X_train,X_test,y_train, y_test):
    
    models = [
        LinearRegression(),
        Ridge(),
        Lasso(),
        DecisionTreeRegressor(),
        RandomForestRegressor(),
        SVR(),
        XGBRegressor(),
        LGBMRegressor(verbose=-1),
        CatBoostRegressor(verbose=False)
    ]
    
    results = []
    for model in models:
        model_name = model.__class__.__name__

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        rmse = mean_squared_error(y_test, y_pred, squared=False)

        r2 = r2_score(y_test, y_pred)
        scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
        rmse_cv = np.sqrt(-scores.mean())
        print(f"{model_name} = RMSE: {rmse}, RMSEcv : {rmse_cv}, r2 Score: {r2}")

In [ ]:
#modelleme ve en uygun modeli bulalım
compare_models(X_train,X_test,y_train, y_test)


In [ ]:
#en uygun modeli tahmin edelim
model = CatBoostRegressor(verbose=False)
model.fit(X_train, y_train,verbose=False)
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2Score = r2_score(y_test, y_pred)
print(f" RMSE: {rmse} \n  r2: {r2Score}")

In [ ]:
#tahminleme sonuçlarını görelim
model_predict_actual=pd.DataFrame({'Gerçek':y_test,'Tahmin':y_pred})
model_predict_actual

In [ ]:
#tahminleme sonuçlarını görelim
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, color='navy', alpha=0.3, label='Tahmin Değerler')
plt.scatter(y_test, y_test, color='yellow', alpha=0.4, label='Gerçek Değerler')
plt.xlabel('Gerçek Değerler')
plt.ylabel('Tahmin Değerler')
plt.title('Gerçek vs Tahmin')
plt.legend()
plt.show()